In [1]:
#!/usr/bin/env python3.6
# -*- coding: utf-8 -*-

In [2]:
# Imports
import sys
import math
import numpy as np
from matplotlib import animation
from IPython.display import HTML
from matplotlib import pyplot as plt
plt.rcParams['animation.ffmpeg_path'] = '/usr/bin/ffmpeg'
import mpl_toolkits.mplot3d.axes3d as p3

In [3]:
np.random.seed(20)
np.set_printoptions(threshold=sys.maxsize)

In [4]:
%matplotlib inline

# Data

In [5]:
# Read data
path = '../../../dataset4_wbc/live_extraction/aliengo/step_0.065/accelerations_1m.csv'
motion = np.genfromtxt(path, delimiter=',', dtype=np.float64)
contacts = np.genfromtxt(path, delimiter=',', usecols=(29, 30, 31, 32), dtype=bool)
print("Motion data has a length of ", len(motion))

Motion data has a length of  13718


In [6]:
print(motion.shape)

(13718, 33)


# Dataset Preparation

In [7]:
import sklearn
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [8]:
def quaternion_rotation_matrix(Q):
    """
    Covert a quaternion into a full three-dimensional rotation matrix.
 
    Input
    :param Q: A 4 element array representing the quaternion (q0,q1,q2,q3) 
 
    Output
    :return: A 3x3 element matrix representing the full 3D rotation matrix. 
             This rotation matrix converts a point in the local reference 
             frame to a point in the global reference frame.
    """
    # Extract the values from Q
    q0 = Q[3]
    q1 = Q[0]
    q2 = Q[1]
    q3 = Q[2]
     
    # First row of the rotation matrix
    r00 = 2 * (q0 * q0 + q1 * q1) - 1
    r01 = 2 * (q1 * q2 - q0 * q3)
    r02 = 2 * (q1 * q3 + q0 * q2)
     
    # Second row of the rotation matrix
    r10 = 2 * (q1 * q2 + q0 * q3)
    r11 = 2 * (q0 * q0 + q2 * q2) - 1
    r12 = 2 * (q2 * q3 - q0 * q1)
     
    # Third row of the rotation matrix
    r20 = 2 * (q1 * q3 - q0 * q2)
    r21 = 2 * (q2 * q3 + q0 * q1)
    r22 = 2 * (q0 * q0 + q3 * q3) - 1
     
    # 3x3 rotation matrix
    rot_matrix = np.array([[r00, r01, r02],
                           [r10, r11, r12],
                           [r20, r21, r22]])
                            
    return rot_matrix

In [10]:
def absolute_footstep_accelerations_dataset(dataset, footsteps, debug=False):
    # Footsteps sizes
    inputs = []
    labels = []
    
    for idx, footstep in enumerate(footsteps):
        if idx > 0:
            # Compute time difference between
            # curhent and next footsteps
            time_difference = abs(dataset[footsteps[idx], 0] - dataset[footsteps[idx-1], 0])

            if time_difference < 0.5:            
                fl_moving = contacts[idx, 0]
                fr_moving = contacts[idx, 1]
                rl_moving = contacts[idx, 2]
                rr_moving = contacts[idx, 3]
                
                assert fr_moving == rl_moving
                assert rr_moving == fl_moving
                assert fr_moving != fl_moving

                # Compute rotation matrices from odometry orientation
                R_prev = quaternion_rotation_matrix(dataset[footsteps[idx-1], 19:23])
                R_curr = quaternion_rotation_matrix(dataset[footsteps[idx], 19:23])

                # Odom base vectors
                prev_base = np.array([[dataset[footsteps[idx-1], 16], dataset[footsteps[idx-1], 17], 0]])
                curr_base = np.array([[dataset[footsteps[idx], 16], dataset[footsteps[idx], 17], 0]])

                # Rotate odometry estimation
                rotated_prev_base = R_prev.T @ prev_base.T
                rotated_curr_base = R_prev.T @ curr_base.T
                base_displacement = rotated_curr_base - rotated_prev_base

                inputs.append(dataset[footsteps[idx-1], 1:4].tolist() +
                              dataset[footsteps[idx-1], 23:29].tolist() +
                              dataset[footsteps[idx-1], 4:16].tolist() + 
                              [fl_moving, fr_moving, rl_moving, rr_moving] +
                              dataset[footsteps[idx], 1:4].tolist())
                    
                if fl_moving and rr_moving:
                    # FL vectors
                    prev_fl = np.array([[dataset[footsteps[idx-1], 4], dataset[footsteps[idx-1], 5], 0]])
                    curr_fl = np.array([[dataset[footsteps[idx], 4], dataset[footsteps[idx], 5], 0]])

                    # RR vectors
                    prev_rr = np.array([[dataset[footsteps[idx-1], 13], dataset[footsteps[idx-1], 14], 0]])
                    curr_rr = np.array([[dataset[footsteps[idx], 13], dataset[footsteps[idx], 14], 0]])

                    # Rotate FL vector
                    rotated_prev_fl = R_prev.T @ prev_fl.T
                    rotated_curr_fl = R_prev.T @ curr_fl.T
                    fl_displacement = rotated_curr_fl - rotated_prev_fl

                    # Rotate RR vector
                    rotated_prev_rr = R_prev.T @ prev_rr.T
                    rotated_curr_rr = R_prev.T @ curr_rr.T
                    rr_displacement = rotated_curr_rr - rotated_prev_rr

                    #labels.append([base_displacement[0,0] + fl_displacement[0,0], base_displacement[1,0] + fl_displacement[1,0],
                    #               base_displacement[0,0] + rr_displacement[0,0], base_displacement[1,0] + rr_displacement[1,0]])    
                    labels.append([fl_displacement[0,0], fl_displacement[1,0], rr_displacement[0,0], rr_displacement[1,0]])  
                else:
                    # FR vectors
                    prev_fr = np.array([[dataset[footsteps[idx-1], 7], dataset[footsteps[idx-1], 8], 0]])
                    curr_fr = np.array([[dataset[footsteps[idx], 7], dataset[footsteps[idx], 8], 0]])

                    # RL vectors
                    prev_rl = np.array([[dataset[footsteps[idx-1], 10], dataset[footsteps[idx-1], 11], 0]])
                    curr_rl = np.array([[dataset[footsteps[idx], 10], dataset[footsteps[idx], 11], 0]])

                    # Rotate FR vector
                    rotated_prev_fr = R_prev.T @ prev_fr.T
                    rotated_curr_fr = R_prev.T @ curr_fr.T
                    fr_displacement = rotated_curr_fr - rotated_prev_fr

                    # Rotate RL vector
                    rotated_prev_rl = R_prev.T @ prev_rl.T
                    rotated_curr_rl = R_prev.T @ curr_rl.T
                    rl_displacement = rotated_curr_rl - rotated_prev_rl

                    #labels.append([base_displacement[0,0] + fr_displacement[0,0], base_displacement[1,0] + fr_displacement[1,0],
                    #               base_displacement[0,0] + rl_displacement[0,0], base_displacement[1,0] + rl_displacement[1,0]])
                    labels.append([fr_displacement[0,0], fr_displacement[1,0], rl_displacement[0,0], rl_displacement[1,0]])
                    
    # Convert to numpy array
    inputs_numpy = np.array(inputs)
    labels_numpy = np.array(labels)
    
    return inputs_numpy, labels_numpy

# Stack datasets
X_footsteps_accelerations_absolute, Y_footsteps_accelerations_absolute = absolute_footstep_accelerations_dataset(motion, np.array([x for x in range(len(motion))]))

# Log
print(X_footsteps_accelerations_absolute.shape)
print(Y_footsteps_accelerations_absolute.shape)

(13679, 28)
(13679, 4)


In [11]:
%store X_footsteps_accelerations_absolute
%store Y_footsteps_accelerations_absolute

Stored 'X_footsteps_accelerations_absolute' (ndarray)
Stored 'Y_footsteps_accelerations_absolute' (ndarray)


In [11]:
Y_footsteps_accelerations_absolute

array([[ 3.98075091e-02, -1.23593874e-03,  3.96313048e-02,
        -1.04226651e-03],
       [ 3.78379819e-02,  5.39553788e-04,  3.81475894e-02,
         1.07390738e-04],
       [ 3.65573112e-02,  5.75602482e-04,  3.63828110e-02,
         6.10778365e-04],
       [ 3.92847749e-02, -1.91664790e-04,  3.76363465e-02,
         3.40388856e-04],
       [ 6.41272427e-02,  2.58915938e-03,  6.41851572e-02,
         2.38634840e-03],
       [ 6.30575379e-02,  5.09072371e-04,  6.12186417e-02,
         9.98452515e-04],
       [ 6.46381527e-02,  1.95994901e-03,  6.43269356e-02,
         1.86185253e-03],
       [ 6.33723325e-02, -2.25453479e-04,  6.10094805e-02,
         1.06425923e-03],
       [ 6.42865440e-02,  1.85587926e-03,  6.40167558e-02,
         1.76955047e-03],
       [ 3.86977469e-02,  1.98336247e-03,  3.69197914e-02,
         2.42557030e-03],
       [ 3.67600649e-02,  2.45012235e-03,  3.63678805e-02,
         2.13749667e-03],
       [ 4.06227703e-02,  1.06399255e-03,  3.88456613e-02,
      

In [10]:
max(Y_footsteps_accelerations_absolute[:, 0])

0.2653922341140742

In [11]:
min(Y_footsteps_accelerations_absolute[:, 0])

-0.10640164084099979

In [17]:
x = np.where(X_footsteps_accelerations_absolute[:, 1] > 0)[0]

In [21]:
Y_footsteps_accelerations_absolute[x, 0]

array([-0.02941428, -0.02897092, -0.02818003, -0.02851726, -0.04370904,
       -0.04612951, -0.04400004, -0.04536078, -0.04492819, -0.02777271,
       -0.02664406, -0.02706976, -0.02630886, -0.0279021 , -0.04224387,
       -0.0457484 , -0.04208046, -0.04396159, -0.04359314, -0.02762623,
       -0.0253431 , -0.02760785, -0.02638165, -0.0271788 , -0.04054026,
       -0.04522881, -0.0405634 , -0.04124891, -0.04241746, -0.02800343,
       -0.0236514 , -0.02525913, -0.02503609, -0.02733618, -0.04055247,
       -0.04515113, -0.04046295, -0.04310527, -0.04197985, -0.02791896,
       -0.02393347, -0.02575112, -0.02484614, -0.02743581, -0.04095054,
       -0.04534015, -0.04088149, -0.04588248, -0.03988587, -0.02924166,
       -0.02367206, -0.02636268, -0.02342821, -0.02757459, -0.04091545,
       -0.04527312, -0.04145914, -0.04146895, -0.04005854, -0.02922727,
       -0.02342269, -0.02827185, -0.02429063, -0.02547836, -0.02559691,
       -0.0455928 , -0.04248915, -0.04346038, -0.04233973, -0.04